In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential

In [2]:
def load_videos(video_path, sequence_length=20):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, sequence_length, dtype=int)

    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (80, 80)) / 255.0  # Resize & normalize
            frames.append(frame)
    cap.release()
    return np.array(frames) if len(frames) == sequence_length else None


In [3]:
def prepare_dataset(dataset_path):
    X, y = [], []
    for label in os.listdir(dataset_path):
        action_path = os.path.join(dataset_path, label)
        if os.path.isdir(action_path):
            for video_file in os.listdir(action_path):
                video_path = os.path.join(action_path, video_file)
                frames = load_videos(video_path)
                if frames is not None:
                    X.append(frames)
                    y.append(label)

    le = LabelEncoder()
    y = le.fit_transform(y)
    y = to_categorical(y)
    return np.array(X), np.array(y), le.classes_


In [5]:
dataset_path = r"C:\Users\sahit\Downloads\project_dataset-20250201T092348Z-001\project_dataset"
X, y, class_names = prepare_dataset(dataset_path)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


MemoryError: Unable to allocate 2.93 MiB for an array with shape (20, 80, 80, 3) and data type float64

In [ ]:

from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import DenseNet121  # Or a different DenseNet version

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(80, 80, 3))
base_model.trainable = False  # Freeze weights


base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Optionally, you can summarize the model
base_model.summary()



In [ ]:
def extract_features(frames):
    features = []
    for frame in frames:
        feature_map = base_model.predict(np.expand_dims(frame, axis=0))  # Shape: (1, 3, 3, 2048)
        pooled_features = GlobalAveragePooling2D()(feature_map)  # Shape: (1, 2048)
        features.append(pooled_features.numpy().flatten())  # Convert tensor to NumPy array
    return np.array(features)  # Shape: (20, 2048)


In [ ]:
X_train = np.array([extract_features(video) for video in tqdm(X_train, desc="Processing training videos")])
X_test = np.array([extract_features(video) for video in tqdm(X_test, desc="Processing testing videos")])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense, Input

model = Sequential([
    Input(shape=(20, 1024)),  # Explicitly define the input shape
    GRU(256, return_sequences=False),  # No need for input_shape here
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Optionally, you can summarize the model
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=8)

In [ ]:
model.save("gru_model.keras")  # Saves in the new format


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

In [ ]:
loss, ac = model.evaluate(X_train, y_train)
print(f"Train Accuracy: {acc * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback

class TrainingPlotCallback(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_data = []

    def on_epoch_end(self, epoch, logs=None):
        # Capture metrics at the end of each epoch
        self.epoch_data.append({
            'accuracy': logs.get('accuracy') * 100,
            'loss': logs.get('loss'),
            'val_accuracy': logs.get('val_accuracy') * 100,
            'val_loss': logs.get('val_loss')
        })

    def plot(self):
        """Plot the captured metrics after training."""
        epochs = range(1, len(self.epoch_data) + 1)
        train_acc = [data['accuracy'] for data in self.epoch_data]
        train_loss = [data['loss'] for data in self.epoch_data]
        val_acc = [data['val_accuracy'] for data in self.epoch_data]
        val_loss = [data['val_loss'] for data in self.epoch_data]

        # Set plot style
        plt.style.use('ggplot')  # Using a built-in style that works

        # Create the plot with two subplots: Accuracy and Loss
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        # Accuracy plot
        ax[0].plot(epochs, train_acc, label='Train Accuracy', color='royalblue', marker='o', markersize=6, linestyle='-', linewidth=2)
        ax[0].plot(epochs, val_acc, label='Validation Accuracy', color='darkorange', marker='s', markersize=6, linestyle='--', linewidth=2)
        ax[0].set_title('Model Accuracy', fontsize=14)
        ax[0].set_xlabel('Epochs', fontsize=12)
        ax[0].set_ylabel('Accuracy', fontsize=12)
        ax[0].legend()
        ax[0].grid(True)

        # Loss plot
        ax[1].plot(epochs, train_loss, label='Train Loss', color='green', marker='^', markersize=6, linestyle='-', linewidth=2)
        ax[1].plot(epochs, val_loss, label='Validation Loss', color='red', marker='x', markersize=6, linestyle='--', linewidth=2)
        ax[1].set_title('Model Loss', fontsize=14)
        ax[1].set_xlabel('Epochs', fontsize=12)
        ax[1].set_ylabel('Loss', fontsize=12)
        ax[1].legend()
        ax[1].grid(True)

        plt.tight_layout()
        plt.show()

# Example usage:
# Assuming you are training a Keras model like this:

# Initialize the custom callback
plot_callback = TrainingPlotCallback()

# Train the model and pass the callback to capture epoch data
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32, callbacks=[plot_callback])

# After training is done, plot the graphs
plot_callback.plot()


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import DenseNet121  # Or a different DenseNet version

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(80, 80, 3))
base_model.trainable = False  # Freeze weights
global_avg_pool = GlobalAveragePooling2D()

# Class names (Make sure it's the same as during training)
class_names = ["Sit down","Fencing","Basketball","Billiards","Sitting","ThrowDiscus",
               "BaseballPitch","Walking","Standing","Stand up","Lying Down",
               "Fall Down","HorseRiding","Drumming","CleanAndJerk",
               "Biking","BenchPress"]  # Change this to match your dataset

# Function to process a single video
def predict_video(video_path, sequence_length=20):
    frames = []
    cap = cv2.VideoCapture(video_path)
    
    while len(frames) < sequence_length:
        ret, frame = cap.read()
        if not ret:
            break  # Stop if video ends

        frame = cv2.resize(frame, (80, 80)) / 255.0  # Resize & normalize
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        print("Error: No frames found in video!")
        return None

    # If the video is too short, repeat the last frame
    while len(frames) < sequence_length:
        frames.append(frames[-1])

    frames = np.array(frames[:sequence_length])

    features = np.array([
        global_avg_pool(base_model.predict(np.expand_dims(frame, axis=0))).numpy().flatten()
        for frame in frames
    ])  # Shape: (20, 2048)

    features = np.expand_dims(features, axis=0)  # Shape: (1, 20, 2048) for model input

    # Predict action
    predictions = model.predict(features)
    predicted_class = np.argmax(predictions)

    print(f"Predicted Action: {class_names[predicted_class]}")
    return class_names[predicted_class]



In [ ]:
# Test with a video file
video_file = r"C:\Users\sahit\Downloads\project_dataset-20250201T092348Z-001\project_dataset\Drumming\v_Drumming_g24_c06.avi"# Change to your test video
predicted_action = predict_video(video_file)

In [ ]:
#excess code

import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load Pretrained DenseNet121 for Feature Extraction
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(80, 80, 3))
base_model.trainable = False  # Freeze the base model
global_avg_pool = GlobalAveragePooling2D()

# Load the trained GRU classification model
model = load_model("gru_model.keras")  # Ensure this file exists

# Define class names (Ensure this matches the model's training labels)
class_names = ["Sit down", "Fencing", "Basketball", "Billiards", "Sitting", "ThrowDiscus",
               "BaseballPitch", "Walking", "Standing", "Stand up", "Lying Down",
               "Fall Down", "HorseRiding", "Drumming", "CleanAndJerk",
               "Biking", "BenchPress"]

# Function to process a video and predict the action
def predict_video(video_path, sequence_length=20):
    frames = []
    cap = cv2.VideoCapture(video_path)
    
    while len(frames) < sequence_length:
        ret, frame = cap.read()
        if not ret:
            break  # Stop if the video ends

        frame = cv2.resize(frame, (80, 80)) / 255.0  # Resize & normalize
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        print("Error: No frames found in video!")
        return None

    # Pad sequence if it's too short
    while len(frames) < sequence_length:
        frames.append(frames[-1])  # Repeat the last frame

    frames = np.array(frames[:sequence_length])  # Convert to NumPy array

    # **Efficient Feature Extraction using Batch Processing**
    features = base_model.predict(frames, batch_size=sequence_length)  # Shape: (20, 2, 2, 1024)
    features = np.array([global_avg_pool(feature[np.newaxis, ...]).numpy().flatten() for feature in features])  
    features = np.expand_dims(features, axis=0)  # Shape: (1, 20, 1024)

    # Predict Action using the GRU Model
    predictions = model.predict(features)
    predicted_class = np.argmax(predictions)

    print(f"Predicted Action: {class_names[predicted_class]}")
    return class_names[predicted_class]

# Example Usage
# predicted_action = predict_video("example_video.mp4")
